In [1]:
from stable_baselines3 import DQN 
import numpy as np
import sys
from collections import OrderedDict
sys.path.insert(0,'boptestGymService')
from boptestGymEnv import BoptestGymEnv
from boptestGymEnv import BoptestGymEnvRewardWeightCost, NormalizedActionWrapper, NormalizedObservationWrapper, SaveAndTestCallback,DiscretizedActionWrapper
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure
from testing import utilities
import random
import os
from stable_baselines3 import SAC,PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.logger import configure
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env

import requests
url = 'http://127.0.0.1:5000'
# url="https://api.boptest.net" 


In [2]:
import numpy as np
import requests

class BoptestGymEnvCustomReward(BoptestGymEnv):
    
    def calculate_objective(self, kpis):
        """
        Calculate the objective based on the given KPI values.
        """
        cost_tot = kpis.get('cost_tot', 0) or 0
        pdih_tot = kpis.get('pdih_tot', 0) or 0
        pele_tot = kpis.get('pele_tot', 0) or 0
        tdis_tot = kpis.get('tdis_tot', 0) or 0
        idis_tot = kpis.get('idis_tot', 0) or 0

        objective = (
            cost_tot +
            4.25 * (pdih_tot + pele_tot) +
            0.005 * tdis_tot +
            0.0001 * idis_tot
        )

        return objective

    def get_reward(self):
        try:
            #use this one running on local server
            kpis = requests.get(f'{self.url}/kpi').json()['payload']

            #use this when running boptest server
            # print(self.test_id)
            # print(self.url)
            # kpis = requests.get('{0}/kpi/{1}'.format(self.url,self.testid)).json()['payload']
            # print(kpis)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching KPIs: {e}")
            return 0  # In case of error, return zero reward

        current_objective = self.calculate_objective(kpis)
        # Compute reward
        
        reward = -(current_objective - self.objective_integrand)
        if current_objective==self.objective_integrand or  reward==-0.0:
            print("Lottery")
            print("prev",self.objective_integrand)
            print("curr",current_objective)
            
            reward=1
            print("reward",reward)
            return reward
        
        print("prev",self.objective_integrand)
        print("curr",current_objective)
        print("reward",reward)
        self.objective_integrand = current_objective
        
        
        return reward
        # self.step_count += 1

        # if self.objective_integrand is not None:
        #     # Calculate the increase in objective
        #     increase = current_objective - self.objective_integrand
        #     print("current objective",current_objective)
        #     print("previos objective",self.objective_integrand)
        #     # Update cumulative increase
        #     self.cumulative_increase += abs(increase)

        #     # Calculate average absolute increase up to this point
        #     average_increase = self.cumulative_increase / self.step_count

        #     # Calculate relative performance (closer to 0 is better)
        #     relative_performance = increase / (average_increase + self.epsilon)

        #     # Transform relative performance to a reward between -1 and 1
        #     reward = -np.tanh(relative_performance)

        #     # Add a small positive reward if increase is less than average
        #     if abs(increase) < average_increase:
        #         reward += 0.1

        #     # Add a larger reward if we managed to decrease the objective
        #     if increase < 0:
        #         reward += 0.5

        # else:
        #     reward = 0
        # print("reward",reward)
        # # Update the stored previous objective
        # self.objective_integrand = current_objective

        # return reward
    # def reset(self):
    #     self.previous_objective = None
    #     self.cumulative_increase = 0
    #     self.step_count = 0
    #     return super().reset()  

In [3]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.logger import configure

def train_PPO_with_callback(model_path=None,
                            log_dir=os.path.join('results', 'PPO_AD2', 'Model4'),
                            tensorboard_log=os.path.join('results', 'PPO_AD2', 'Model4')):
    """
    Method to train a PPO agent using a callback to save the model periodically.

    Parameters
    ----------
    model_path : str, optional
        Path to a pre-trained model. If provided, the model will be loaded and further trained.
    log_dir : str
        Directory where monitoring data and best-trained model are stored.
    tensorboard_log : str
        Path to directory to load tensorboard logs.
    """
    
    excluding_periods = []
    excluding_periods.append((173*24*3600, 266*24*3600))  # Summer period

    env = BoptestGymEnvCustomReward(
        url=url,
        actions=['hvac_oveAhu_yFan_u'],
        observations = OrderedDict([
                            ('time', (0, 31536000)),
                            ('weaSta_reaWeaTDryBul_y', (250., 310.)), # Outdoor temperature range
                            ('hvac_reaZonCor_CO2Zon_y', (0, 2000)),  # CO2 range in ppm
                            ('hvac_reaZonEas_CO2Zon_y', (0, 2000)),
                            ('hvac_reaZonNor_CO2Zon_y', (0, 2000)),
                            ('hvac_reaZonSou_CO2Zon_y', (0, 2000)),
                            ('hvac_reaZonWes_CO2Zon_y', (0, 2000)),
                            ('hvac_reaZonCor_TZon_y', (270, 310)),  # Temperature in K
                            ('hvac_reaZonEas_TZon_y', (270, 310)),
                            ('hvac_reaZonNor_TZon_y', (270, 310)),
                            ('hvac_reaZonSou_TZon_y', (270, 310)),
                            ('hvac_reaZonWes_TZon_y', (270, 310))
                            ]),
        scenario={'electricity_price': 'highly_dynamic'},
        random_start_time=True,
        max_episode_length=3*24*3600,
        warmup_period=1*24*3600,
        step_period=3600,
        log_dir=tensorboard_log,
        excluding_periods=excluding_periods
    )

    print(env.observation_space)
    # env = NormalizedObservationWrapper(env)
    # env = NormalizedActionWrapper(env)  
    env = DiscretizedActionWrapper(env,n_bins_act=15)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    
    env = Monitor(env=env, filename=os.path.join(log_dir, 'monitor.csv'))
    
    # Callback to save model every 2000 steps
    callback = SaveAndTestCallback(check_freq=48,save_freq=500,env=env,log_dir=tensorboard_log)
    
    # Set up logger with TensorBoard logging continuation
    new_logger = configure(log_dir, ['stdout', 'csv', 'tensorboard'])
    
    # Load existing model if model_path is given, else create a new one
    if model_path and os.path.isfile(model_path):
        model = PPO.load(model_path, env=env, tensorboard_log=tensorboard_log)
        print(f"Loaded pre-trained model from {model_path}")
        model.set_logger(new_logger)  # Reconfigure the logger to continue logging
    else:
        model = PPO(
            'MlpPolicy', 
            env, 
            verbose=1, 
            gamma=0.99,
            learning_rate=3e-4,
            n_steps=2048,
            batch_size=64,
            n_epochs=10,
            clip_range=0.2,
            gae_lambda=0.95,
            ent_coef=0.01,
            tensorboard_log=tensorboard_log,
        )
        model.set_logger(new_logger)
        print("Starting training from scratch.")
    
    # Train the agent with the callback
    model.learn(total_timesteps=int(500000), callback=callback)
    
    return env, model

if __name__ == "__main__":
    model_path=None
    # model_path = "results/PPO/Model4/ppo_model_120000_latest_final.zip" # Update this with the correct path if needed
    env, model = train_PPO_with_callback(model_path=model_path)
    model.save(os.path.join('results', 'PPO', 'final_model'))
    print("Training completed. Model saved in results/PPO/")
    print("TensorBoard logs saved in results/PPO/")


Box([  0. 250.   0.   0.   0.   0.   0. 270. 270. 270. 270. 270.], [3.1536e+07 3.1000e+02 2.0000e+03 2.0000e+03 2.0000e+03 2.0000e+03
 2.0000e+03 3.1000e+02 3.1000e+02 3.1000e+02 3.1000e+02 3.1000e+02], (12,), float32)
Logging to results/PPO_AD2/Model4


/home/aniket/miniconda3/envs/boptestlocal/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Using cuda device
Wrapping the env in a DummyVecEnv.
Starting training from scratch.
u {'hvac_oveAhu_yFan_u': 0.13333334028720856, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0
curr 0.005526272924477026
reward -0.005526272924477026
u {'hvac_oveAhu_yFan_u': 0.800000011920929, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0.005526272924477026
curr 0.03251195251626204
reward -0.026985679591785012
u {'hvac_oveAhu_yFan_u': 0.5333333611488342, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0.03251195251626204
curr 0.03278136617635995
reward -0.00026941366009790996
u {'hvac_oveAhu_yFan_u': 0.6000000238418579, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0.03278136617635995
curr 0.033053056649543194
reward -0.00027169047318324757
u {'hvac_oveAhu_yFan_u': 0.13333334028720856, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0.033053056649543194
curr 0.03306230628831577
reward -9.2496387725724e-06
u {'hvac_oveAhu_yFan_u': 0.7333333492279053, 'hvac_oveAhu_yFan_activate': 1.0}
prev 0.03306230628831577
curr 0.033198851841031066
rewar

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))